In [151]:
import spotipy
import spotipy.util as util
import pandas as pd

CLIENT_ID = "73b813922c7c4b5e96f2fa2cc2245c1b"
CLIENT_SECRET = "ebb3ec0a8cca4cddbe86f65edf1530f8"

token = util.spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

/Users/dangause/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  if __name__ == '__main__':


In [152]:
cache_token

'BQDufK0HuKQFqalRNZiThSQHmg-XxRaEYLLPUDyPdYkX33hXnaK7c1XgF2746VXefsgj5_xuupTxmcjGiMA'

In [135]:
# this is a function we will use that "turns" the spotify pages in a playlist, otherwise max pl size = 100
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name","track_id","genre","popularity","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = get_playlist_tracks(creator, playlist_id)
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # genre
        if len(sp.artist(track["track"]["artists"][0]["external_urls"]["spotify"])["genres"]) > 0:
            genres = sp.artist(track["track"]["artists"][0]["external_urls"]["spotify"])["genres"]
            genre_string = ""
            for genre in genres:
                genre_string = genre_string + genre + ", "
            playlist_features["genre"] = genre_string[:-2]
        else: playlist_features["genre"] = "n/a"
        #popularity
        playlist_features["popularity"] = track["track"]["popularity"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [136]:
analyze_playlist("Thomas Thaxton", "5lQjLY1dikUaX0ppw5wj7I")

,artist,album,track_name,track_id,genre,popularity,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,General Ghost,If Then,If Then,41tazZ7j9pJdT5x5ZvIQQI,n/a,0,0.697,0.8060,6,-4.944,1,0.0638,0.000142,0.0946,0.5150,146.970,229440,4
1,Sufjan Stevens,Illinois,The Predatory Wasp Of The Palisades Is Out To ...,6pRVZFlSom0481gI0cXQNU,"baroque pop, chamber pop, freak folk, indie fo...",0,0.490,0.4200,4,-10.721,1,0.0365,0.126000,0.1100,0.3250,153.119,323013,3
2,"Now, Now",Threads,Dead Oaks,3x7youRVqZ0GPIOAJEEXv8,"dreamo, electropop, indie poptimism, indie roc...",0,0.656,0.6630,9,-8.208,1,0.0405,0.000374,0.0870,0.6270,155.957,101040,4
3,Darwin Deez,Darwin Deez,Bad Day,7hUuGmq5C0AlB4X58TrQ3e,new rave,0,0.788,0.3950,2,-8.521,1,0.1240,0.000000,0.1670,0.8080,105.985,186107,4
4,Volcano Choir,Repave,Byegone,4Jpy9tZRmbmpbMSWIRlDyR,"eau claire indie, indie folk, wisconsin indie",0,0.496,0.3520,0,-10.280,1,0.0357,0.037400,0.1180,0.1020,140.010,262533,4
5,Lotte Kestner,China Mountain,Lula Boat,27X1Qfb7WhKEynsTOkE8LG,indie psych-pop,23,0.658,0.1610,9,-15.778,1,0.0298,0.001330,0.1190,0.1800,79.997,226107,4
6,Milky Chance,Sadnecessary,Fairytale,4F0ptEqZa9ZbTu8t4g5FuR,german pop,51,0.756,0.8060,7,-9.188,0,0.0309,0.162000,0.1010,0.9380,106.023,256000,4
7,Hey Marseilles,Lines We Trace (Deluxe Edition),Heart Beats,4vWnuvWn9DNQU99mqPtp6d,stomp and holler,0,0.535,0.5340,7,-8.021,1,0.0343,0.034800,0.1150,0.1840,125.910,251107,4
8,Kay Kay and His Weathered Underground,Kay Kay and His Weathered Underground,Hey Momma',2f8SgIR9bKJX7FKRvJU7uO,dreamo,36,0.575,0.5310,6,-7.666,0,0.0420,0.000499,0.1790,0.4900,89.965,250892,4
9,Wake Owl,Wild Country EP,Gold,2xbmLKnfGYTkzWRUX3DUpG,"stomp and holler, vancouver indie",24,0.483,0.5870,11,-8.813,0,0.0296,0.003460,0.1200,0.3890,86.018,224093,4


In [ ]:
## csv
#df.to_csv("dataframe.csv", index = False)
## excel
#df.to_excel("dataframe.xlsx", index = False)

In [122]:
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        playlist_df = analyze_playlist(*val)
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [137]:
playlist_dict = {
    'ultimate_indie' : ('Guardian Music', '7sF6XyjiKKbi9HIcCgzvgh'), 
    'underground_hits' : ('spotify', '37i9dQZF1DX30HHrCAl4ZG'),
    'super_swag' : ('Thomas Thaxton', '5lQjLY1dikUaX0ppw5wj7I'),
    'indie_pop' : ('spotify', '37i9dQZF1DWWEcRhUVtL8n')
}

## Then by running the following code we can see the result
multiple_playlist_df = analyze_playlist_dict(playlist_dict)
multiple_playlist_df["playlist"].value_counts()


# will also need to remove duplicates

super_swag          306
indie_pop           100
ultimate_indie       80
underground_hits     52
Name: playlist, dtype: int64

In [138]:
len(multiple_playlist_df)

538